In [73]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F 

# In previous the 8 tokens are not talking to each other, we used to biagram to predict next word using n-1 word 
## so here our aim make this tokes talks with each other 
torch.manual_seed(1337)

B, T, C = 4, 8, 2 
x = torch.randn(B, T, C) 
x.shape


## for this we can calculate the average of previuous numbers like if you are in 5th element you can calculate average for first 5 element 
## [0, 1, 2, 3, 4], you are in fifth element so the average is 2.0 and the current element is 5 the average can convey some of the information 
## of previous element. Let's try to achieve this in code 

torch.Size([4, 8, 2])

In [48]:
## averaging until current element to get the history of previous elements 
a = [1, 2, 3, 4, 5, 6, 7, 8 , 9]

for i in range(len(a)): 
    avg = sum(a[:i+1]) / len(a[:i+1])
    print(a[:i+1])
    print(avg)


## Here we are averaging the information, but this is very inefficient manner because we need to do this for batching so 2 for loop may come 


xbow = torch.zeros( (B, T, C))  # Batch size, tensor lenght, channels 
for b in range(B): # Because we need to do the average for all the batches 
    for t in range(T): ## we are iterating to the ith batch 
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)


## In-efficient loop here :) You can check by comparing x[0] and xbow[0]

## we can do this more efficient by using matrix multiplications by using tril (triangular)

[1]
1.0
[1, 2]
1.5
[1, 2, 3]
2.0
[1, 2, 3, 4]
2.5
[1, 2, 3, 4, 5]
3.0
[1, 2, 3, 4, 5, 6]
3.5
[1, 2, 3, 4, 5, 6, 7]
4.0
[1, 2, 3, 4, 5, 6, 7, 8]
4.5
[1, 2, 3, 4, 5, 6, 7, 8, 9]
5.0


In [66]:
wei = torch.tril(torch.ones(T, T)) 
wei = wei / wei.sum(1, keepdim=True) 

xbow2 = wei @ x # (T, T) @ (B, T, C)  # this is batched matrix multiply (this is more efficient)
# compare xbow1 and xbow2 (both are same only)

In [78]:
# let's re-write in some more way 
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim =-1)
wei = wei @ x 
wei

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

[1]
1.0
[1, 2]
1.5
[1, 2, 3]
2.0
[1, 2, 3, 4]
2.5
[1, 2, 3, 4, 5]
3.0
[1, 2, 3, 4, 5, 6]
3.5
[1, 2, 3, 4, 5, 6, 7]
4.0
[1, 2, 3, 4, 5, 6, 7, 8]
4.5
[1, 2, 3, 4, 5, 6, 7, 8, 9]
5.0


In [37]:
(0+ 1+2+3+4) / 5

2.0